# EER Method
use EER method to automatically generate threshold that has good separability

In [1]:
from scipy.spatial.distance import euclidean
from sklearn.metrics import accuracy_score, f1_score
from keras import backend as K
from feature_extractor.face_feature_extractor import FaceFeatureExtractor
import numpy as np
import cv2 as cv
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import os
import time

Using TensorFlow backend.


In [2]:
DATA_DIR_PATH = '../exp_data/'
root_paths = []
file_paths = []
for root, dirs, files in os.walk(DATA_DIR_PATH):
     for file in files:
        root_paths.append(root)
        file_paths.append(os.path.join(root, file))

In [3]:
len(root_paths)

90

# Experiment

In [4]:
mtcnn_detector = MTCNN()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [5]:
feature_extractor = FaceFeatureExtractor(None, extractor_name='vgg_face_resnet50')

In [6]:
def extract_image_feature(face_detector, feature_extractor, file_path):
    img = cv.cvtColor(cv.imread(file_path), cv.COLOR_BGR2RGB)
    detection_result = face_detector.detect_faces(img)
    cropped_image = None
    feature = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    if cropped_image is not None:
        feature = feature_extractor.extract_image(cropped_image)
    return feature

In [7]:
feature = extract_image_feature(mtcnn_detector, feature_extractor, file_paths[2])
print(feature)

None


In [28]:
def count_tp_tn_fp_fn(threshold, root_paths, file_paths):
    # positive is new
    # negative is old
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i in range(len(root_paths) - 1):
        for j in range(i + 1, len(root_paths)):
            # Add more condition to handle imbalance
            if file_paths[j].split('/')[-1] == file_paths[i].split('/')[-1] or root_paths[i] == root_paths[j]:
                feature1 = extract_image_feature(mtcnn_detector, feature_extractor, file_paths[i])
                feature2 = extract_image_feature(mtcnn_detector, feature_extractor, file_paths[j])
                if feature1 is not None and feature2 is not None:
                    distance = euclidean(feature1, feature2)
                    if distance >= threshold: # diff person
                        if root_paths[i] != root_paths[j]:
                            tp += 1
                        else:
                            fp += 1
                    else: # same person
                        if root_paths[i] == root_paths[j]:
                            tn += 1
                        else:
                            fn += 1
#             else:
#                 if feature2 is None: # the one that want to be predicted
                    
#                 else: # feature 1 is None
    return (tp, tn, fp, fn)

### Harusnya dari 35 - 165

In [29]:
result = {}
for i in range(35, 170, 5):
    result[i] = count_tp_tn_fp_fn(i, root_paths, file_paths)
    print(result[i])

(306, 0, 346, 0)
(306, 6, 340, 0)
(306, 17, 329, 0)
(306, 39, 307, 0)
(306, 71, 275, 0)
(306, 104, 242, 0)
(306, 135, 211, 0)
(306, 159, 187, 0)
(306, 200, 146, 0)
(306, 226, 120, 0)
(306, 251, 95, 0)
(305, 273, 73, 1)
(303, 284, 62, 3)
(299, 300, 46, 7)
(285, 315, 31, 21)
(271, 329, 17, 35)
(247, 336, 10, 59)
(221, 341, 5, 85)
(169, 345, 1, 137)
(130, 346, 0, 176)
(83, 346, 0, 223)
(48, 346, 0, 258)
(21, 346, 0, 285)
(7, 346, 0, 299)
(2, 346, 0, 304)
(1, 346, 0, 305)
(0, 346, 0, 306)


## EER

In [35]:
far_list = []
frr_list = []

In [36]:
for key, value in result.items():
    tp, tn, fp, fn = value
    far = fp / (fp+tn)
    frr = fn / (tp+fn)
    far_list.append(far)
    frr_list.append(frr)

In [37]:
counter = 40
for i in range(len(frr_list)):
    print(counter, far_list[i], frr_list[i])
    counter += 5

40 1.0 0.0
45 0.9826589595375722 0.0
50 0.9508670520231214 0.0
55 0.8872832369942196 0.0
60 0.7947976878612717 0.0
65 0.6994219653179191 0.0
70 0.6098265895953757 0.0
75 0.5404624277456648 0.0
80 0.42196531791907516 0.0
85 0.3468208092485549 0.0
90 0.2745664739884393 0.0
95 0.21098265895953758 0.0032679738562091504
100 0.1791907514450867 0.00980392156862745
105 0.1329479768786127 0.02287581699346405
110 0.08959537572254335 0.06862745098039216
115 0.049132947976878616 0.11437908496732026
120 0.028901734104046242 0.19281045751633988
125 0.014450867052023121 0.2777777777777778
130 0.002890173410404624 0.4477124183006536
135 0.0 0.5751633986928104
140 0.0 0.7287581699346405
145 0.0 0.8431372549019608
150 0.0 0.9313725490196079
155 0.0 0.9771241830065359
160 0.0 0.9934640522875817
165 0.0 0.9967320261437909
170 0.0 1.0


In [38]:
result

{35: (306, 0, 346, 0),
 40: (306, 6, 340, 0),
 45: (306, 17, 329, 0),
 50: (306, 39, 307, 0),
 55: (306, 71, 275, 0),
 60: (306, 104, 242, 0),
 65: (306, 135, 211, 0),
 70: (306, 159, 187, 0),
 75: (306, 200, 146, 0),
 80: (306, 226, 120, 0),
 85: (306, 251, 95, 0),
 90: (305, 273, 73, 1),
 95: (303, 284, 62, 3),
 100: (299, 300, 46, 7),
 105: (285, 315, 31, 21),
 110: (271, 329, 17, 35),
 115: (247, 336, 10, 59),
 120: (221, 341, 5, 85),
 125: (169, 345, 1, 137),
 130: (130, 346, 0, 176),
 135: (83, 346, 0, 223),
 140: (48, 346, 0, 258),
 145: (21, 346, 0, 285),
 150: (7, 346, 0, 299),
 155: (2, 346, 0, 304),
 160: (1, 346, 0, 305),
 165: (0, 346, 0, 306)}